In [21]:
import requests
import pandas as pd

# Define base URL and dataset name
BASE_URL = "https://data.melbourne.vic.gov.au/api/v2/catalog/datasets/"
#City_dataset = "city-activities-and-planned-works"
City_dataset = "bicycle-network"

# Set the number of records to fetch
NUMBER_OF_RECORDS = 10

# Add a filter for the number of results returned
FILTERS = f'records?limit={NUMBER_OF_RECORDS}&offset=0&timezone=UTC'

# Fetch data from the API
api_url = BASE_URL + City_dataset + '/' + FILTERS
response = requests.get(api_url)

if response.status_code == 200:
    json_data = response.json()
    records = json_data['records']
    print("Data fetched successfully!")
else:
    print(f"Error: {response.status_code}")

Data fetched successfully!


#### Explanation: 
- We are importing the requests library to fetch data from the City of Melbourne API and pandas library to manipulate and analyze the data.
- The base URL and dataset name are set to access the City of Melbourne open data portal API for the bicycle network dataset. The number of records to fetch is set to 10, and the filters are created accordingly.
- We make a GET request to the API using the requests library. If the response status code is 200, it means the data has been fetched successfully, and we parse the JSON data into a Python dictionary.

In [22]:
def get_data(base, data_url, size=0):
    filters = f'records?limit={10}&offset={size}&timezone=UTC'
    url = f'{base}{data_url}/{filters}'
    result = requests.get(url)
    result_json = result.json()
    max_results = result_json['total_count']
    links = result_json['links']
    records = result_json['records']
    records_df = pd.json_normalize(records)

    # Fix columns names
    records_df.drop(columns=['links'], inplace=True)
    column_names = records_df.columns.values.tolist()

    # Replace location.lat & location.lon
    column_names = ['_'.join((a.split(".")[-2:])) if a.split('.')[-2] == 'location' else a for a in column_names]
    column_names = [i.split('.')[-1] for i in column_names]
    records_df.columns = column_names
    
    # Extract latitude and longitude values from coordinates
    records_df['lat'] = records_df['coordinates'].apply(lambda x: x[0][1])
    records_df['lon'] = records_df['coordinates'].apply(lambda x: x[0][0])


    next_url = None

    # Get next url
    if records_df.shape[0] != max_results:
        for l in links:
            if l['rel'] == 'next':
                next_url = l['href']

    return [records_df, next_url, column_names]


#### Explanation: This function fetches the data from the API, normalizes the JSON records, and creates a DataFrame. It also fixes the column names and gets the URL for the next set of records, if available.

In [23]:
df_new, url_next, column_names = get_data(BASE_URL,City_dataset)

#### Explanation: We call the get_data function to fetch the initial data, which returns the DataFrame, the next URL, and the column names.

In [24]:
City_df = pd.DataFrame(columns=column_names)

In [25]:
# Create function to add API output to dataframe
def add_to_dataframe(dataframe, new_data):
    df = pd.concat([dataframe, new_data])
    return df

#### Explanation: The pd.concat method is used to concatenate two dataframes along a particular axis. In this case, it's concatenating the existing dataframe and the new data along the rows axis (i.e., vertically). Finally, the updated dataframe is returned.

In [26]:
# Add initial output to datframe
City_df = add_to_dataframe(City_df, df_new)

In [27]:
# Get the rest of the Dataset
while url_next is not None:
    df_new, url_next, column_names = get_data(BASE_URL, City_dataset, City_df.shape[0])
    City_df = add_to_dataframe(City_df, df_new)
    print(City_df.shape)

(20, 16)
(30, 16)
(40, 16)
(50, 16)
(60, 16)
(70, 16)
(80, 16)
(90, 16)
(100, 16)
(110, 16)
(120, 16)
(130, 16)
(140, 16)
(150, 16)
(160, 16)
(170, 16)
(180, 16)
(190, 16)
(200, 16)
(210, 16)
(220, 16)
(230, 16)
(240, 16)
(250, 16)
(260, 16)
(270, 16)
(280, 16)
(290, 16)
(300, 16)
(310, 16)
(320, 16)


KeyboardInterrupt: 

In [1]:
import pandas as pd

file_path = 'C:/Users/Minh Pro/Downloads/bicycle-network.csv'

def add_to_dataframe(dataframe, new_data):
    df = pd.concat([dataframe, new_data])
    return df

# Read the CSV file into a DataFrame
City_df = pd.read_csv(file_path)

# Apply the filter for the 'direction' column with the value 'F'
filtered_df = City_df[City_df['Direction'] == 'F']


In [2]:
City_df

,Geo Point,Geo Shape,OBJECTID,RoadClass,Width,Oneway,Management,Direction,Shape_Length,Description
0,"-37.80380092845, 144.95392315095","{""coordinates"": [[144.9539178084, -37.80383110...",50001,23.0,2.1,FT,Council Major,F,6.764870,dooring protecting chevron
1,"-37.80401088565, 144.95406331635002","{""coordinates"": [[144.954066723, -37.803991467...",50008,22.0,1.5,FT,Council Major,F,4.352156,Standard bicycle lane
2,"-37.80449600605, 144.9539782171","{""coordinates"": [[144.9539921547, -37.80441655...",50013,22.0,1.5,FT,Council Major,F,17.807399,Standard bicycle lane
3,"-37.804826883299995, 144.95391946465","{""coordinates"": [[144.953928449, -37.804776797...",50017,22.0,1.5,FT,Council Major,F,11.230462,Standard bicycle lane
4,"-37.805002590868135, 144.95388784237431","{""coordinates"": [[144.9539009322, -37.80493019...",50019,22.0,1.5,FT,Council Major,F,16.234070,Standard bicycle lane
...,...,...,...,...,...,...,...,...,...,...
130043,"-37.801907915049995, 144.94903351925","{""coordinates"": [[144.9490257403, -37.80195195...",100988,92.0,0.0,NaN,Connector,B,9.871339,Centroid connector
130044,"-37.78820208955, 144.94334771525","{""coordinates"": [[144.9433145275, -37.78836831...",100990,92.0,0.0,NaN,Connector,B,37.360133,Centroid connector
130045,"-37.81745626875, 144.96644833375","{""coordinates"": [[144.9665434648, -37.81758035...",100992,92.0,0.0,NaN,Connector,B,32.238733,Centroid connector
130046,"-37.8041080622, 144.96547090829998","{""coordinates"": [[144.9653623633, -37.80412131...",100994,92.0,0.0,NaN,Connector,B,19.343050,Centroid connector


In [3]:
import pandas as pd

# Load the dataset from the CSV file
City_df = pd.read_csv('C:/Users/Minh Pro/Downloads/bicycle-network.csv')

# Split 'Geo Point' column into separate latitude and longitude columns
City_df[['lat', 'lon']] = City_df['Geo Point'].str.split(',', expand=True).astype(float)

# Define the latitude and longitude boundaries for St Kilda Road
min_lat, max_lat = -37.855, -37.835
min_lon, max_lon = 144.963, 144.985

# Filter the dataset based on the latitude and longitude boundaries
stkilda_df = City_df[(City_df['lat'] >= min_lat) & (City_df['lat'] <= max_lat) &
                     (City_df['lon'] >= min_lon) & (City_df['lon'] <= max_lon)]

# Analyze the different types of bike lanes and road types
bike_lane_counts = stkilda_df['Description'].value_counts()
road_type_counts = stkilda_df['RoadClass'].value_counts()

# Display the results
print("Bike Lane Counts:")
print(bike_lane_counts)
print("\nRoad Type Counts:")
print(road_type_counts)


Bike Lane Counts:
Description
Centroid connector                                    1935
Entrance connecctor                                    543
Local Road                                             469
Collector Road                                         106
Arterial  Road                                          88
Walking Track                                           75
Standard bicycle lane                                   69
Interesection                                           52
Shared Path                                             25
Shared Intersection (Bike Lane)                         21
Total protecte chevron (protect from dooring and *      19
Sub-Arterial Road                                        8
2wd Road                                                 1
Name: count, dtype: int64

Road Type Counts:
RoadClass
92.0    1935
91.0     543
5.0      469
4.0      106
2.0       88
11.0      75
22.0      69
30.0      52
12.0      25
29.0      21
25.0      19
3.0  

# Conclusion

From our analysis of the bike network data for St Kilda Road, several key insights emerge:

1. **Diverse Bike Network:** The St Kilda Road area boasts a diverse range of bike lane types, with the most common being "Centroid connector," "Entrance connector," and "Local Road." This variety caters to different cycling needs, whether for commuting, leisure, or exercise. 

2. **Prevalence of Connectors:** The prevalence of connectors (both "Centroid" and "Entrance") suggests a bike network designed to link major points of interest efficiently. This infrastructure plays a crucial role in making cycling a viable mode of transportation for many trips.

3. **Room for Improvement:** However, the lower prevalence of certain types of bike lanes, such as "Shared Paths" and "Shared Intersection (Bike Lane)," indicates areas for potential infrastructure improvement. Enhancing these types of facilities could further boost cyclist safety and convenience.

4. **Impact on Cycling Culture:** Overall, the presence and diversity of the bike network on St Kilda Road likely have a significant impact on the local cycling culture. The infrastructure supports and encourages cycling as a mode of transportation, contributing to a healthier, more environmentally friendly community.

5. **Road Class Variety:** The variety in "RoadClass" indicates that the bike network spans different types of roads, further promoting accessibility and connectivity for cyclists.

6. **Bike Network Length:** The total length of the bike network on St Kilda Road is significant, offering ample routes for cyclists and indicating a substantial commitment to bike infrastructure in this area.

In conclusion, the bike network on St Kilda Road appears robust and diverse, contributing positively to the local community's mobility and lifestyle. However, there are opportunities for further improvements, particularly in expanding the less prevalent types of bike lanes. As the city continues to grow and evolve, ongoing analysis of this sort will be crucial in informing decisions about future bike infrastructure development.


In [4]:
import pandas as pd

# Load data from CSV
df = pd.read_csv('C:/Users/Minh Pro/Downloads/bicycle-network.csv')

# Split "Geo Point" into two new columns "Lat" and "Lon"
df[['Lat', 'Lon']] = df['Geo Point'].str.split(',', expand=True)
df[['Lat', 'Lon']] = df[['Lat', 'Lon']].apply(pd.to_numeric)

# Define St Kilda Road bounding box
min_lat, max_lat = -37.85, -37.82
min_lon, max_lon = 144.97, 144.98

# Filter rows for St Kilda Road
stkilda_df = df[(df['Lat'] >= min_lat) & (df['Lat'] <= max_lat) & 
                (df['Lon'] >= min_lon) & (df['Lon'] <= max_lon)]

# Downsample to 1000 random rows
stkilda_df = stkilda_df.sample(20)

# Display the filtered DataFrame
print(stkilda_df)


                                      Geo Point   
87240       -37.835720986300004, 144.9785399329  \
4957    -37.821912672006896, 144.97491803858372   
40559            -37.8355879173, 144.9787935097   
12744           -37.8395215789, 144.97739190865   
111408         -37.82903824105, 144.97015432905   
944      -37.82367887738801, 144.97853317077042   
111894           -37.8221828912, 144.9768371499   
83557    -37.82686731134845, 144.97435174428182   
3196            -37.84666601785, 144.9796546321   
8587       -37.836497517850006, 144.97924512275   
117988            -37.84256571, 144.97850852045   
116904         -37.83562227305, 144.97730555555   
29442   -37.840044903649996, 144.97758345674998   
81028          -37.84334910335, 144.97879289385   
127196          -37.8408281728, 144.97786956525   
45923           -37.8401754613, 144.97763118475   
102411       -37.8447413364, 144.97997783354998   
30232   -37.845777015850004, 144.97964810874998   
112517       -37.8447413364, 14

In [8]:
import pandas as pd

# Load data from CSV
df = pd.read_csv('C:/Users/Minh Pro/Downloads/bicycle-network.csv')

# Split "Geo Point" into two new columns "Lat" and "Lon"
df[['Lat', 'Lon']] = df['Geo Point'].str.split(',', expand=True)
df[['Lat', 'Lon']] = df[['Lat', 'Lon']].apply(pd.to_numeric)

# Define St Kilda Road bounding box
min_lat, max_lat = -37.85, -37.82
min_lon, max_lon = 144.97, 144.98

# Filter rows for St Kilda Road and road description
stkilda_df = df[(df['Lat'] >= min_lat) & (df['Lat'] <= max_lat) & 
                (df['Lon'] >= min_lon) & (df['Lon'] <= max_lon) & 
                (df['Description'] == 'Standard bicycle lane')]

# Display the filtered DataFrame
print(stkilda_df)


                                     Geo Point   
1075    -37.84033949258863, 144.97678192318637  \
1077   -37.832329444799996, 144.97154604119999   
1078           -37.8299680837, 144.97106228975   
1079    -37.84685298434151, 144.97915119810048   
1080   -37.845432763737364, 144.97863670359098   
...                                        ...   
70148           -37.8435999796, 144.9784818567   
70149   -37.84391318882324, 144.97859586256368   
70151    -37.84530511613666, 144.9791034051839   
70152    -37.84628315366829, 144.9794558683008   
70153           -37.8468416091, 144.9796595056   

                                               Geo Shape  OBJECTID  RoadClass   
1075   {"coordinates": [[144.9771327982, -37.84129948...     53146       22.0  \
1077   {"coordinates": [[144.9715581747, -37.83234942...     53164       22.0   
1078   {"coordinates": [[144.9710623197, -37.82997016...     53167       22.0   
1079   {"coordinates": [[144.9794424592, -37.84767896...     53170       22

In [9]:
import folium

# Create a map centered around the middle of the bounding box
m = folium.Map(location=[(min_lat + max_lat) / 2, (min_lon + max_lon) / 2], zoom_start=13)

# Iterate over the rows of the DataFrame
for idx, row in stkilda_df.iterrows():
    # Create a string with the information for the popup
    popup_text = f"RoadClass: {row['RoadClass']}<br>Width: {row['Width']}<br>Oneway: {row['Oneway']}<br>Management: {row['Management']}<br>Direction: {row['Direction']}<br>Shape_Length: {row['Shape_Length']}<br>Description: {row['Description']}"
    
    # Create a marker at the Lat, Lon coordinates with the popup text
    marker = folium.Marker(location=[row['Lat'], row['Lon']], popup=popup_text)
    
    # Add the marker to the map
    marker.add_to(m)

# Display the map
m


## Conclusion

We can draw several conclusions about the impact of the St Kilda Road bike lane:

1. **Safety**: The bike lanes in St Kilda road are predominantly "Standard bicycle lanes". This type of dedicated lane significantly enhances safety for cyclists by giving them their own space, separate from vehicular traffic.

2. **Connectivity and Accessibility**: The continuous bike lane along St Kilda Road suggests excellent connectivity. This feature potentially transforms the road into a major cycling artery, providing efficient and safe transportation for numerous cyclists.

3. **Cycling Promotion**: The city's investment in a prominent bike lane along a significant road like St Kilda Road indicates its commitment to promoting cycling as a sustainable mode of transport. This initiative could motivate more individuals to adopt cycling for commuting, fitness, or recreational purposes.

4. **Environmental Impact**: By establishing and maintaining cycling infrastructure such as the St Kilda Road bike lane, the city promotes environmentally friendly transportation options. This strategy could lead to a decrease in carbon emissions, contributing to a healthier urban environment.

5. **Physical Health and Well-being**: Encouraging active transportation like cycling through these bike lanes can contribute to the physical health and well-being of Melbourne residents. Regular physical activity such as cycling can deliver significant health benefits.


In [10]:
# Calculate total length
total_length = stkilda_df['Shape_Length'].sum()

print(f"The total length of the bike network on St Kilda Road is {total_length} units.")


The total length of the bike network on St Kilda Road is 5773.421801665866 units.


#### This indicates a substantial investment in cycling infrastructure along this corridor, suggesting a commitment to promoting cycling as a viable and sustainable mode of transport.

The presence of such extensive bike network infrastructure can have several potential impacts:

1. **Increased cycling activity**: A well-established network can encourage more people to cycle, either for commuting or recreational purposes. This can help reduce traffic congestion and carbon emissions.

2. **Improved safety**: Dedicated bike lanes and paths can improve safety for cyclists, potentially reducing the risk of accidents.

3. **Health benefits**: By facilitating active transport, the bike network can contribute to the physical health and wellbeing of people who cycle regularly.

4. **Economic benefits**: Bike lanes can also have economic benefits. They can increase the attractiveness of the area for residents and businesses, potentially boosting property values and local economy.

### I repeat the process of creating the table, but this time, it is for bike share dock location. 
=> Bike share dock location data can be useful in the development of St. Kilda's new bike lane network as it provides information on where people are most likely to rent and return bikes, indicating areas of high demand for cycling infrastructure. The location data can also be used to determine where new bike share docks should be placed to improve access to bike share services and encourage cycling as a mode of transportation.

In [2]:
import requests
import pandas as pd

#base_url
BASE_URL = 'https://data.melbourne.vic.gov.au/api/v2/catalog/datasets/'

domain = 'data.melbourne.vic.gov.au'

#Dataset name
City_dataset = 'bike-share-dock-locations'
NUMBER_OF_RECORDS = 10

# Add a filter for the number of results returned
FILTERS = f'records?limit={NUMBER_OF_RECORDS}&offset=0&timezone=UTC'

def get_data(base, data_url, size = 0):
    
    filters = f'records?limit={10}&offset={size}&timezone=UTC'
    url = f'{base}{data_url}/{filters}'
    result = requests.get(url)
    result_json = result.json()
    max_results = result_json['total_count']
    links = result_json['links']
    records = result_json['records']
    records_df = pd.json_normalize(records)
    
    # fix columns names
    records_df.drop(columns=['links'],inplace=True)
    column_names = records_df.columns.values.tolist()
    
    # Replace location.lat & location.lon
    column_names = ['_'.join((a.split(".")[-2:])) if a.split('.')[-2]=='location' else a for a in column_names]
    column_names = [i.split('.')[-1] for i in column_names]
    records_df.columns = column_names
    
    next_url = None
    
    # get next url
    if records_df.shape[0] != max_results:
        for l in links:
            if l['rel']=='next':
                next_url = l['href']
                
    return [records_df, next_url, column_names]

df_new, url_next, column_names = get_data(BASE_URL, City_dataset)
dock_df = pd.DataFrame(columns=column_names)

def add_to_dataframe(dataframe, new_data):
    df = pd.concat([dataframe, new_data])
    return df

dock_df = add_to_dataframe(dock_df, df_new)

while url_next != None:
    df_new, url_next, column_names = get_data(BASE_URL, City_dataset, dock_df.shape[0])
    dock_df = add_to_dataframe(dock_df, df_new)
    print(dock_df.shape)

(20, 11)
(30, 11)
(40, 11)
(50, 11)


In [4]:
dock_df

,id,timestamp,size,station_id,name,rental_method,capacity,lat,lon,location_lon,location_lat
0,31b238e436f855bce0a2e040035d2d24032aef43,2022-11-13T10:05:45.725Z,144,7,Bourke Street Mall - 205 Bourke St - City,"CREDITCARD,KEY",11,-37.813088,144.967437,144.967437,-37.813088
1,cee6ea9564196ed57d942c3a9d97edffbc7c3bbc,2022-11-13T10:05:45.725Z,146,16,NAB - Harbour Esp / Bourke St - Docklands,"CREDITCARD,KEY",33,-37.818306,144.945923,144.945923,-37.818306
2,d764626fef10e15fc2934036dd8266b39b2b50bb,2022-11-13T10:05:45.725Z,144,21,Bridport St / Montague St - Albert Park,"CREDITCARD,KEY",23,-37.840885,144.955303,144.955303,-37.840885
3,0c88265f67290bcd70da2b11fbc8c119210f6a93,2022-11-13T10:05:45.725Z,142,22,Pickles St / Ingles St - Port Melbourne,"CREDITCARD,KEY",23,-37.835803,144.948520,144.948520,-37.835803
4,354e577cf608ab2ac3789c7e2c54f8715f6dd5c2,2022-11-13T10:05:45.725Z,159,24,North Melbourne Station - Adderley St - North ...,"CREDITCARD,KEY",19,-37.807021,144.941854,144.941854,-37.807021
5,cb4e50fc2235b24c5c3eede3430d09bedaf3dc0e,2022-11-13T10:05:45.725Z,129,26,Beach St - Port Melbourne,"CREDITCARD,KEY",39,-37.841954,144.935296,144.935296,-37.841954
6,d9b4830e8320b544e7623e1f0a6978c019b03926,2022-11-13T10:05:45.725Z,146,28,Queensbridge St / Yarra River - Southbank,"CREDITCARD,KEY",15,-37.821071,144.961113,144.961113,-37.821071
7,8c93996d9f7f1e3faca2bff93bb4413ebe815a41,2022-11-13T10:05:45.725Z,139,31,Cleve Gardens - Fitzroy St - St Kilda,"CREDITCARD,KEY",15,-37.861984,144.973410,144.973410,-37.861984
8,de62d805c3de65868a3cb02c845f3010cdfce516,2022-11-13T10:05:45.725Z,158,32,West Beach Pavilion - Beaconsfield Pde - St Ki...,"CREDITCARD,KEY",15,-37.858553,144.969101,144.969101,-37.858553
9,78a011b7ae69829ef7fb663aedcffd65ac14314f,2022-11-13T10:05:45.725Z,147,33,Siddeley St / Seafarers Bridge - Docklands,"CREDITCARD,KEY",23,-37.823124,144.951918,144.951918,-37.823124


In [24]:
# Filter the bike share docks located in St Kilda
stkilda_docks_df = dock_df[dock_df['name'].str.contains('St Kilda Rd')]

# Print the resulting DataFrame
stkilda_docks_df

,id,timestamp,size,station_id,name,rental_method,capacity,lat,lon,location_lon,location_lat,distance_to_stkilda
6,85ebac5af022f38891ddb52446fe30cc0b77d0ff,2022-11-13T10:05:45.725Z,152,47,VCAM - St Kilda Rd / Southbank Blvd - Southbank,"CREDITCARD,KEY",11,-37.824088,144.970085,144.970085,-37.824088,2.167926
2,ea07986384f3492c2ea6eb3a0993a0f4921cf2b0,2022-11-13T10:05:45.725Z,159,41,Domain Interchange - Park St / St Kilda Rd - M...,"CREDITCARD,KEY",15,-37.832396,144.971678,144.971678,-37.832396,1.255306
0,5771786b27932dace77f7e3fa47151365336e7af,2022-11-13T10:05:45.725Z,157,15,Vic Barracks - Coventry St / St Kilda Rd - Sou...,"CREDITCARD,KEY",11,-37.828887,144.970822,144.970822,-37.828887,1.643626
5,f891b8a4043887f9877e61ee40eda35d37c34938,2022-11-13T10:05:45.725Z,140,42,Kings Way / St Kilda Rd - Melbourne,"CREDITCARD,KEY",27,-37.836323,144.975162,144.975162,-37.836323,0.736267


In [25]:
total_capacity = stkilda_docks_df['capacity'].sum()
print(f"Total bike dock capacity on St Kilda Rd: {total_capacity}")


Total bike dock capacity on St Kilda Rd: 64


The total bike dock capacity on St Kilda Rd being 64 means that at any given time, there can be up to 64 bikes available for rent on this route

In [28]:
import numpy as np

# Define the coordinates for your point of interest
poi_lat, poi_lon = -37.8183, 144.9671  

# Calculate the distance from each bike dock to the point of interest using haversine formula
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # radius of Earth in kilometers
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    dist = 2 * R * np.arcsin(np.sqrt(np.sin(delta_phi/2)**2 
                                      + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda/2)**2))
    return dist

stkilda_docks_df.loc[:, 'distance_to_poi'] = haversine(poi_lat, poi_lon, stkilda_docks_df['lat'], stkilda_docks_df['lon'])

# Find the dock that is closest to the point of interest
closest_dock = stkilda_docks_df.loc[stkilda_docks_df['distance_to_poi'].idxmin()]
print(f"Closest dock to the point of interest is at {closest_dock['name']} with a distance of {closest_dock['distance_to_poi']} km")

# Find the dock that is furthest from the point of interest
furthest_dock = stkilda_docks_df.loc[stkilda_docks_df['distance_to_poi'].idxmax()]
print(f"Furthest dock from the point of interest is at {furthest_dock['name']} with a distance of {furthest_dock['distance_to_poi']} km")


Closest dock to the point of interest is at VCAM - St Kilda Rd / Southbank Blvd - Southbank with a distance of 0.694953215279491 km
Furthest dock from the point of interest is at Kings Way / St Kilda Rd - Melbourne with a distance of 2.125476821180524 km


In [29]:
import folium

# Create a map centered around St Kilda
stkilda_map = folium.Map(location=[stkilda_lat, stkilda_lon], zoom_start=14)

# Add a marker for the point of interest
folium.Marker(
    [poi_lat, poi_lon],
    popup='Point of Interest',
    icon=folium.Icon(icon="cloud"),
).add_to(stkilda_map)

# Add a marker for each bike dock
for index, row in stkilda_docks_df.iterrows():
    folium.Marker(
        [row['lat'], row['lon']], 
        popup=f"{row['name']}<br>Distance to POI: {row['distance_to_poi']:.2f} km"
    ).add_to(stkilda_map)

# Display the map
stkilda_map


#### By using this information, we can provide users with a list of nearby bike share stations in St Kilda, their capacities, and their locations (latitude and longitude coordinates). This can help users plan their bike share trips more effectively, and also aid in the development of new bike lanes and bike share stations in the St Kilda area based on demand and usage data.